In [1]:
# Improved implementation with connection established outside the function
import oracledb

# Establish connection outside the function (do this once)
def create_oracle_connection():
    try:
        connection = oracledb.connect(
            user="map", 
            password="mapper", 
            dsn="172.16.20.69:1521/demodb"
        )
        # print("Oracle connection established successfully")
        return connection
    except Exception as e:
        print(f"Error establishing Oracle connection: {str(e)}")
        raise


In [18]:
def call_create_update_job(connection, p_mapref):
    """
    Calls the Oracle function CREATE_UPDATE_JOB
    
    Args:
        connection: Oracle connection object
        p_mapref: Mapping reference to create/update job
    
    Returns:
        tuple: A tuple containing (job_id, error_message) where:
            - job_id: The returned job ID from the function
            - error_message: Any error message returned by the Oracle function
    """
    cursor = None
    try:
        cursor = connection.cursor()
        
        # Define the output parameter for job_id
        v_job_id = cursor.var(oracledb.NUMBER)  # Assuming jobid is a NUMBER in Oracle
        
        # SQL to execute with named parameters
        sql = """
        BEGIN
            :job_id := MAP.PKGDWJOB.CREATE_UPDATE_JOB(
                p_mapref => :p_mapref
            );
        END;
        """
        
        # Execute with named parameters
        cursor.execute(
            sql,
            job_id=v_job_id,
            p_mapref=p_mapref
        )
        connection.commit()
        
        # Get the result
        job_id = v_job_id.getvalue()
        
        return job_id, None
    
    except Exception as e:
        error_message = f"Error creating/updating job: {str(e)}"
        print(error_message)
        return None, error_message
    
    finally:
        if cursor:
            cursor.close()

# Example usage
if __name__ == "__main__":
    # This would be replaced with your actual connection code
    # import oracledb
    connection = create_oracle_connection()
    
    mapping_ref = "MAP001"  # Example mapping reference
    
    # Call the function
    job_id, error = call_create_update_job(connection, mapping_ref)
    
    # Display the results
    if error:
        print(f"Error: {error}")
    else:
        print(f"Job created/updated successfully. Job ID: {job_id}")
    
    connection.close()

Error creating/updating job: ORA-20099: PKGDWJOB - CREATE_UPDATE_JOB:-
299 : MAP001
 Err:ORA-20099: PKGDWJOB - CREATE_UPDATE_JOB:-
206 : MAP001
 Err:ORA-20099: PKGDWJOB - CREATE_TARGET_TABLE:-
199 : MAP001
 Err:ORA-20099: PKGDWJOB - CREATE_TARGET_TABLE:-
102 : MAP001
 Err:ORA-01918: user 'DW_SCHSD' does not exist
ORA-06512: at "MAP.PKGERR", line 30
ORA-06512: at "MAP.PKGDWJOB", line 325
ORA-06512: at line 2
Error: Error creating/updating job: ORA-20099: PKGDWJOB - CREATE_UPDATE_JOB:-
299 : MAP001
 Err:ORA-20099: PKGDWJOB - CREATE_UPDATE_JOB:-
206 : MAP001
 Err:ORA-20099: PKGDWJOB - CREATE_TARGET_TABLE:-
199 : MAP001
 Err:ORA-20099: PKGDWJOB - CREATE_TARGET_TABLE:-
102 : MAP001
 Err:ORA-01918: user 'DW_SCHSD' does not exist
ORA-06512: at "MAP.PKGERR", line 30
ORA-06512: at "MAP.PKGDWJOB", line 325
ORA-06512: at line 2


In [15]:
import oracledb

def call_activate_deactivate_mapping(connection, p_mapref, p_stflg):
    """
    Calls the Oracle procedure ACTIVATE_DEACTIVATE_MAPPING
    
    Args:
        connection: Oracle connection object
        p_mapref: Mapping reference to activate/deactivate
        p_stflg: Status flag ('A' to activate, 'N' to deactivate)
    
    Returns:
        tuple: A tuple containing (success, message) where:
            - success: Boolean indicating if the operation was successful
            - message: Success or error message
    """
    cursor = None
    try:
        cursor = connection.cursor()
        
        # Define the output parameter for error message
        p_err = cursor.var(oracledb.STRING, 2000)  # Assuming error message can be up to 2000 chars
        
        # SQL to execute with named parameters
        sql = """
        BEGIN
            PKGDWMAPR.ACTIVATE_DEACTIVATE_MAPPING(
                p_mapref => :p_mapref,
                p_stflg => :p_stflg,
                p_err => :p_err
            );
        END;
        """
        
        # Execute with named parameters
        cursor.execute(
            sql,
            p_mapref=p_mapref,
            p_stflg=p_stflg,
            p_err=p_err
        )
        connection.commit()
        
        # Get the error message (if any)
        error_message = p_err.getvalue()
        
        if error_message:
            return False, f"Error: {error_message}"
        else:
            action = "activated" if p_stflg == "A" else "deactivated"
            return True, f"Mapping {p_mapref} successfully {action}"
    
    except Exception as e:
        error_message = f"Exception while activating/deactivating mapping: {str(e)}"
        return False, error_message
    
    finally:
        if cursor:
            cursor.close()

# Example usage
if __name__ == "__main__":
    # This would be replaced with your actual connection code
    connection = create_oracle_connection()
    
    mapping_ref = "MAP001"  # Example mapping reference
    status_flag = "A"       # 'A' to activate, 'N' to deactivate
    
    # Call the procedure
    success, message = call_activate_deactivate_mapping(connection, mapping_ref, status_flag)
    
    # Display the results
    print(message)
    
    connection.close()

Mapping MAP001 successfully activated


In [16]:
success

True